In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display
import pandas as pd

section_dict = {'최신뉴스':'BreakingNews','프리뷰':'Preview','스타 인터뷰':'Interview','KBO PHOTO':'KboPhoto'}

def save_news(dict, section_name):    
    m_id = section_dict[section_name]

    url = f'https://www.koreabaseball.com/MediaNews/News/{m_id}/List.aspx'

    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }

    res = requests.get(url, headers=req_header)

    if res.ok:
        res.encoding = 'utf-8'
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        
        if m_id=='KboPhoto':
            tags = soup.select("ul.photoList.pl40 li")

            for i, li in enumerate(tags, 1):
                news_data = {}
                
                a_tag = li.find('a')
                if a_tag:
                    link = urljoin(url, a_tag['href'])
                    news_data['link'] = link

                img_tag = li.select_one('img')
                if img_tag and img_tag.get('src'):
                    img_url = urljoin(url, img_tag['src'])
                    news_data['img'] = img_url
                    # display(Image(url=img_url))
                    
                title_tag = li.select_one('span')
                if title_tag:
                    title = title_tag.text.strip()
                    news_data['title'] = title
                    
                dict.append(news_data)

        else:
            tags = soup.select("ul.boardPhoto li")

            for i, li in enumerate(tags, 1):
                news_data = {}
                
                a_tag = li.find('a')
                if a_tag:
                    link = urljoin(url, a_tag['href'])
                    news_data['link'] = link

                img_tag = li.select_one('img')
                if img_tag and img_tag.get('src'):
                    img_url = urljoin(url, img_tag['src'])
                    news_data['img'] = img_url
                    # display(Image(url=img_url))
                    
                title_tag = li.select_one('strong')
                if title_tag:
                    title = title_tag.text.strip()
                    news_data['title'] = title
                    
                dict.append(news_data)

    else:
        print(f'에러코드 = {res.status_code}')

In [ ]:
new_dict = []
preview_dict = []
interview_dict = []
photo_dict = []

save_news(new_dict, '최신뉴스')
save_news(preview_dict, '프리뷰')
save_news(interview_dict, '스타 인터뷰')
save_news(photo_dict, 'KBO PHOTO')

new_df = pd.DataFrame(new_dict)
preview_df = pd.DataFrame(preview_dict)
interview_df = pd.DataFrame(interview_dict)
photo_df = pd.DataFrame(photo_dict)

new_df.to_csv('../../csv/scrap_news_new.csv', index=False)
preview_df.to_csv('../../csv/scrap_news_preview.csv', index=False)
interview_df.to_csv('../../csv/scrap_news_interview.csv', index=False)
photo_df.to_csv('../../csv/scrap_news_photo.csv', index=False)
